## NYC 311 Service Request Data Analysis using SQLAlchemy

### Questions

1. What fraction of complaints are associated with the 2nd most popular agency?
2. What is the most 'surprising' complaint type when conditioned on a borough? That is, what is the largest ratio of the conditional probability of a complaint type given a specified borough divided by the unconditioned probability of that complaint type?
3. What is the distance (in degrees) between the 90% and 10% percentiles of degrees latitude?
4. Let's estimate the area that 311 supports. Suppose calls are 2D normally distributed on
the surface of the earth with mean and standard deviation given by those of the latitude and longitude. How many square kilometers is the single-standard-deviation ellipse?  
5. What is the difference between the expected number of calls received during the most and
least popular whole hours of the day? 
6. What is the standard deviation in seconds of the time between consecutive calls?

##### Data Columns

- <font color=green>Unique Key: (int)</font>
- <font color=green>Created Date: (datetime)</font>
- </font>Closed Date: (datetime)</font>
- <font color=green>Agency: (str)</font>
- <font color=green>Agency Name: (str)</font>
- <font color=green>Complaint Type: (str)</font>
- Descriptor: (str)
- Location Type: (str
- Incident Zip: (int)
- Incident Address: (int)
- Street Name: (str)
- Cross Street 1: (str)
- Cross Street 2: (str)
- Intersection Street 1: (str)
- Intersection Street 2: (str)
- Address Type: (str)
- City: (str)
- Landmark: (str)
- Facility Type: (str)
- Status: (str)
- Due Date: (str)
- Resolution Description: (str)
- Resolution Action Updated Date: (str)
- Community Board: (str)
- <font color=green>Borough: (str)</font>
- X Coordinate (State Plane): (int)
- Y Coordinate (State Plane): (int)
- Park Facility: (unspecified)
- Park Borough: (str)
- School Name: (str)
- School Number: (unspecified)
- School Region: (unspecified)
- School Code: (unspecified)
- School Phone Number: (int)
- School Address: (str)
- School City: (str)
- School State: (str)
- School Zip: (int)
- School Not Found: 
- School or Citywide Complaint:
- Vehicle Type:
- Taxi Complaint Borough:
- Taxi Pick Up Location: (str)
- Bridge Highway Name: (str)
- Bridge Highway Direction: 
- Road Ramp:
- Bridge Highway Segment:
- Garage Lot Name:
- Ferry Direction:
- Ferry Terminal Name:
- <font color=green>Latitude: (float)</font>
- <font color=green>Longitude: (float)</font>
- Location: (str/float)

In [14]:
import pandas as pd
pd.set_option('display.max_columns', 75)
pd.set_option('display.max_rows', 250)

import numpy as np

from math import pi, radians, degrees, cos, sin, asin, sqrt
import datetime as dt
import time

# Import SQLAlchemy
#! pip install -- upgrade sqlalchemy
from sqlalchemy import create_engine # database connection

In [2]:
# View files in the folder
!ls

Data_Challenge-5.pdf      Project-Proposal.ipynb    book-5-17-03.pdf
NYC_311_Calls.db          RandomNumbers.pdf         computational_physics.pdf
Prob-1.ipynb              RandomWalk.pdf            lectures2012.pdf
Prob-2.ipynb              artist_data.csv           painting_url.csv
Problem-1.py              artist_data_scraped.txt   pricePerCountry.csv
Problem-2.py              avgPricePerArtist.csv     srwbook.pdf
Problem-3.py              avgPricePerCountry.csv


In [3]:
# Number of lines in csv
!wc -l < 311_Service_Requests_from_2010_to_Present.csv 

/bin/sh: 311_Service_Requests_from_2010_to_Present.csv: No such file or directory


In [4]:
# Initialize database with file name "NYC_311_Calls.db" 
disk_engine = create_engine('sqlite:///NYC_311_Calls.db')

```python
# To calculate processing time 
start = dt.datetime.now()

# Size of *.csv chunk to process
chunk_size = 25000

# Counter to keep track of processing time
j = 0

index_start = 1

for df in pd.read_csv('311_Service_Requests_from_2010_to_Present.csv', chunksize = chunk_size, 
                      iterator = True, encoding = 'utf-8'):
    # Rename dataframe columns: by removing space from column names
    df = df.rename(columns = {c: c.replace(' ', '') for c in df.columns})
    
    # Convert column data to datetimes
    df['CreatedDate'] = pd.to_datetime(df['CreatedDate'])
    df['ClosedDate'] = pd.to_datetime(df['ClosedDate'])
    df['DueDate'] = pd.to_datetime(df['DueDate'])
    df['ResolutionActionUpdatedDate'] = pd.to_datetime(df['ResolutionActionUpdatedDate'])
    
    # List of columns to keep
    cols = ['UniqueKey', 'CreatedDate', 'ClosedDate', 'Agency', 'ComplaintType', 'DueDate', 
            'ResolutionActionUpdatedDate', 'Borough', 'Latitude', 'Longitude']
    
    # Drop columns that are not in the list of columns to keep
    for c in df.columns:
        if c not in cols:
            df = df.drop(c, axis = 1)
    
    
    # Print: No. of rows processed
    j += 1
    t = (dt.datetime.now() - start).seconds
    r = j * chunk_size # Rows processed
    print '{} seconds: processed {} rows'.format(t, r)
    
    # Write to an SQL database
    df.to_sql('data', disk_engine, if_exists = 'append')
    
    index_start = df.index[-1] + 1
    
# 6.35 GB to 2 GB in 3 hours!


# Create a dictionary 
df_dict = dict(zip(df['col_name-1'], df['col_name-2']))
```

In [5]:
# Fraction of complaints associated with 2nd most popular agency
df = pd.read_sql_query('SELECT Agency, COUNT(*) as Num_Complaints FROM data GROUP BY Agency ORDER BY Num_Complaints DESC'
                       , disk_engine)

total_complaints = df['Num_Complaints'].sum()
most_pop_2 = df.ix[1, 'Num_Complaints']
frac = most_pop_2/float(total_complaints)

print 'Fraction of complaints associated with 2nd most popular agency: {}'.format(frac)

Fraction of complaints associated with 2nd most popular agency: 0.171329057923


In [6]:
# Complaint Type Count
ct = pd.read_sql_query('SELECT ComplaintType, COUNT(*) as Num_Complaints FROM data GROUP BY ComplaintType ORDER BY Num_Complaints DESC'
                       , disk_engine)

# Unique Key, Complaint Type and Borough
df = pd.read_sql_query('SELECT UniqueKey, ComplaintType, Borough FROM data', disk_engine)

# Group By Complaint Type and Borough
cond = pd.DataFrame({'Count': df.groupby(['ComplaintType', 'Borough'])['UniqueKey'].count()}).reset_index()

# Calculate unconditioned probability of Complaint Types
ct['Probability'] = ct['Num_Complaints']/float(df.shape[0])
del ct['Num_Complaints']

# Use Pivot to reshape dataframe in matrix format
cond_new = cond.pivot(index = 'ComplaintType', columns = 'Borough', values = 'Count').reset_index()
# Fill NaN with 0
cond_new.fillna(0, inplace = True)
# Now calculate: P(ComplaintType_i|Borough_j) = P(ComplaintType_i, Borough_j)/P(Borough_j)
for col in cond_new.columns.tolist()[1:]:
    cond_new[col] = cond_new[col]/float(cond_new[col].sum())

# Merge Conditional probability dataframe with unconditional dataframe    
ratio = pd.merge(cond_new, ct, left_on = 'ComplaintType', right_on = 'ComplaintType', how = 'inner') 
max_ratio = []
for col in ratio.columns.tolist()[1:-1]:
    ratio[col] = ratio[col]/ratio['Probability']
    max_ratio.append(np.max(ratio[col]))

print 'Largest ratio of the conditional probability of a complanint type given',\
      'a specified borough divided by the unconditional probability of that',\
      'complaint type is {}'.format(np.max(max_ratio))

Largest ratio of the conditional probability of a complanint type given a specified borough divided by the unconditional probability of that complaint type is 18.2793463908


In [7]:
# Distance (in degrees) between the 90% and 10% percentiles of degrees latitude
df = pd.read_sql_query('SELECT Latitude FROM data', disk_engine)

# Get 10th and 90th percentile
lat_10 = df['Latitude'].quantile(0.1)
lat_90 = df['Latitude'].quantile(0.9)

# Get Mean and Std. Dev
lat_mean = df['Latitude'].mean()
lat_std = df['Latitude'].std()

In [8]:
# Distance (in degrees) between the 90% and 10% percentiles of degrees longitude
df = pd.read_sql_query('SELECT Longitude FROM data', disk_engine)

# Get 10th and 90th percentile
lon_10 = df['Longitude'].quantile(0.1)
lon_90 = df['Longitude'].quantile(0.9)

# Get Mean and Std. Dev
lon_mean = df['Longitude'].mean()
lon_std = df['Longitude'].std()

In [9]:
def haversine(theta):
    '''
    Calculate haversine of an angle (theta in radians)
    '''
    out = sin(theta/2.0) * sin(theta/2.0)
    return out

def distance(point_1, point_2, mile = False):
    '''
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees): point_1 = (lat1, lon1), point_2 = (lat2, lon2)
    Use mile = True for calculating distance in miles
    Ref: https://en.wikipedia.org/wiki/Haversine_formula
    '''
    # Convert decimal degrees to radians 
    phi1 = radians(point_1[0])
    phi2 = radians(point_2[0])
    lam1 = radians(point_1[1])
    lam2 = radians(point_2[1])
    # Haversine formula 
    dphi = phi2 - phi1 
    dlam = lam2 - lam1
    h = haversine(dphi) + cos(phi1) * cos(phi2) * haversine(dlam)
    if mile: # If mile is True: Calculate distance in miles
        r = 3956 # Radius of earth in miles
        d = 2 * r * asin(sqrt(h))
    else: # Calculate distance in kilometers
        r = 6371 # Radius of earth in kilometers
        d = 2 * r * asin(sqrt(h))
    return d

# Prepare points as per condition in problem
center = (lat_mean, lon_mean) 
lon_dir = (lat_mean, lon_mean + lon_std)
lat_dir = (lat_mean + lat_std, lon_mean)

# Calculate length of major and minor axes of ellipse
a = distance(center, lon_dir)
b = distance(center, lat_dir)

# Calculate area of ellipse
area = pi * a * b
print 'Area (Sq. km) of single-standard-deviation ellipse is {}'.format(area)

Area (Sq. km) of single-standard-deviation ellipse is 209.88505083


In [12]:
# Calculate distance (in degrees) between the 90% and 10% percentiles of degrees longitude
# Since information on longitude is not given, I am using mean longitude

def distance_deg(point_1, point_2):
    '''
    Calculate the great circle distance in degrees between two points 
    on the earth (specified in decimal degrees): point_1 = (lat1, lon1), point_2 = (lat2, lon2)
    Ref: https://en.wikipedia.org/wiki/Haversine_formula
    '''
    # Convert decimal degrees to radians 
    phi1 = radians(point_1[0])
    phi2 = radians(point_2[0])
    lam1 = radians(point_1[1])
    lam2 = radians(point_2[1])
    # Haversine formula 
    dphi = phi2 - phi1 
    dlam = lam2 - lam1
    h = haversine(dphi) + cos(phi1) * cos(phi2) * haversine(dlam)
    deg = degrees(2 * asin(sqrt(h)))
    return deg

# Calculate distance in degrees between (lat10%, mean_lon) and (lat90%, mean_lon)
pt1 = (lat_10, lon_mean)
pt2 = (lat_90, lon_mean)

deg_dist = distance_deg(pt1, pt2)
print 'Distance (in degrees) between 10th and 90th percentiles of degree latitude is {}'.format(deg_dist)


## Check
# 1. Calculate distance in km between (lat10%, mean_lon) and (lat90%, mean_lon)
km_dist = distance(pt1, pt2)

# 2. Convert km_dist to degrees
r = 6371 # Radius of earth in km
km_deg_dist = km_dist * 360 /(2 * pi * r)
print '-------------------------------------------------------------------------'
print km_deg_dist

Distance (in degrees) between 10th and 90th percentiles of degree latitude is 0.235746664383
-------------------------------------------------------------------------
0.235746664383


In [13]:
# Most and least popular hours, and time between consecutive calls
df = pd.read_sql_query('SELECT UniqueKey, CreatedDate, strftime("%H", CreatedDate) as Hour FROM data'
                       , disk_engine)

In [15]:
# Begin to calculate processing time
start = dt.datetime.now()

# Convert object to datetime
df['CreatedDate'] = df['CreatedDate'].apply(lambda t: pd.to_datetime(t))

# Calculate elapsed time since processing began
elapsed = (dt.datetime.now() - start).seconds
print 'Total Processing time: {}'.format(elapsed)

Total Processing time: 783


In [16]:
## Removing points which do not seem to accurately reflect the actual time they were reported
# Create a new temporary column by shifting CreatedDate column 1 row up
df['Date_Before'] = df['CreatedDate'].shift(-1)

# Check if current CratedDate >= next CreatedDate
condition = df['CreatedDate'] >= df['Date_Before'].shift(-1)
del df['Date_Before']

print 'Number of bad points: {}'.format(df.shape[0]-sum(condition))

# Remove points that do not meet condition
df = df[condition]

Number of bad points: 9


In [17]:
# Standard Deviation of time between Consecutive Call
calls = pd.DataFrame({'Date': df.ix[:, 1]})

# Calculate time between consecutive calls
calls['Time'] = calls['Date'] - calls['Date'].shift(-1)

# Get seconds from numpy.timedelta64 type and calculate standard deviation
calls['Seconds'] = calls['Time']/np.timedelta64(1, 's')
std_dev = calls['Seconds'].std()
print 'Standard Deviation (sec) of time between consecutive calls: {}'.format(std_dev)

Standard Deviation (sec) of time between consecutive calls: 57.5007674336


In [18]:
# Get day from Timestamp
df['Day'] = df['CreatedDate'].apply(lambda d: d.day)

In [19]:
# Groupy Day and Hour, then get counts
hour = pd.DataFrame({'Count': df.groupby(['Day', 'Hour'])['UniqueKey'].count()}).reset_index()
hour.head(3)

,Day,Hour,Count
0,1,00,116482
1,1,01,3162
2,1,02,2529


In [20]:
# Use Pivot to reshape dataframe in matrix format
hr_count = hour.pivot(index = 'Hour', columns = 'Day', values = 'Count').reset_index()
cols = hr_count.columns.tolist()[1:]
hr_count['ExpectedCalls'] = hr_count[cols].mean(axis = 1)
diff = hr_count['ExpectedCalls'].max() - hr_count['ExpectedCalls'].min()
print 'Difference between the expected number of calls received during the most and least,\
popular hours of the day: {}'.format(diff)

Difference between the expected number of calls received during the most and least,popular hours of the day: 116019.83871


## Answers

- Fraction of complaints associated with 2nd most popular agency: 0.171329057923  
- Largest ratio of the conditional probability of a complanint type given a specified borough divided by the unconditional probability of that complaint type is 18.2793463908  
- Distance (in degrees) between 10th and 90th percentiles of degree latitude is 0.235746664383
- Area (Sq. km) of single-standard-deviation ellipse is 209.88505083
- Difference between the expected number of calls received during the most and least popular hours of the day: 116019.83871
- Standard Deviation (sec) of time between consecutive calls: 57.5007674336